In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import pytz
import glob
from dateutil.parser import isoparse
from datetime import datetime, timedelta
from DirectoryGenerator import DirectoryGenerator

In [ ]:
dirGen = DirectoryGenerator()

In [ ]:
startDate = datetime.strptime(str(input("Start Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
startDate

In [ ]:
endDate = datetime.strptime(str(input("End Date (Format: YYYY-MM-DD): ")), "%Y-%m-%d").astimezone(pytz.utc)
endDate

In [ ]:
totalDays = abs(endDate - startDate).days + 1
totalDays

In [ ]:
def parseTimestamp(timestamp):
    try:
        return isoparse(timestamp).astimezone(pytz.utc)
    except:
        return np.nan

In [ ]:
discussionForumsDataPostsFilesPath = str(input())

In [ ]:
discussionForumsDataPostsFiles = glob.glob(discussionForumsDataPostsFilesPath + dirGen.getDelimiter() + '**' + dirGen.getDelimiter() + '*.csv', recursive=True)

In [ ]:
discussionForumsPostsData = pd.DataFrame()
for discussionForumsDataPostsFile in discussionForumsDataPostsFiles:
    discussionForumsPostsData = discussionForumsPostsData.append(pd.read_csv(discussionForumsDataPostsFile)[['metadata_event_time', 'metadata_context_id', 'metadata_context_role', 'metadata_user_id', 'body_assignment_id', 'body_discussion_topic_id', 'body_parent_discussion_entry_id']], ignore_index = True)
    
discussionForumsPostsData

In [ ]:
discussionForumsPostsData['metadata_event_time'] = discussionForumsPostsData['metadata_event_time'].apply(parseTimestamp)
discussionForumsPostsData.dropna(subset=['metadata_event_time'], inplace = True)
discussionForumsPostsData

In [ ]:
discussionForumsPostsData = discussionForumsPostsData.loc[(discussionForumsPostsData['metadata_event_time'] >= startDate) & (discussionForumsPostsData['metadata_event_time'] <= endDate)]
discussionForumsPostsData

In [ ]:
discussionForumsPostsData = discussionForumsPostsData.loc[discussionForumsPostsData['metadata_context_role'] == 'StudentEnrollment']
discussionForumsPostsData.drop(['metadata_context_role'], axis = 1, inplace = True)
discussionForumsPostsData

In [ ]:
discussionForumsPostsData.drop(['body_parent_discussion_entry_id'], axis = 1, inplace = True)
discussionForumsPostsData

In [ ]:
def isGradedDiscussionForum(idNum):
    try:
        return int(str(idNum).split("_")[-1]) > 0
    except(AttributeError):
        return False
    
discussionForumsPostsData['is_graded'] = discussionForumsPostsData['body_assignment_id'].apply(isGradedDiscussionForum)
discussionForumsPostsData

In [ ]:
courseTopicGradedUngradedCheck = discussionForumsPostsData.groupby(['metadata_context_id', 'body_discussion_topic_id', 'is_graded'])['metadata_event_time'].count().to_frame('total').reset_index()
courseTopicGradedUngradedCheck

In [ ]:
courseTopicGradedUngradedCheck['course_topic'] = courseTopicGradedUngradedCheck['metadata_context_id'] + "-" + courseTopicGradedUngradedCheck['body_discussion_topic_id']
courseTopicGradedUngradedCheck = courseTopicGradedUngradedCheck.drop(['metadata_context_id', 'body_discussion_topic_id', 'total'], axis = 1)
courseTopicGradedUngradedCheck

In [ ]:
courseTopicGradedUngradedAmbiguous = courseTopicGradedUngradedCheck.groupby(['course_topic'])['is_graded'].count().to_frame('total').reset_index()
courseTopicGradedUngradedAmbiguous

In [ ]:
courseTopicGradedUngradedAmbiguous = courseTopicGradedUngradedAmbiguous.loc[courseTopicGradedUngradedAmbiguous['total'] > 1]
courseTopicGradedUngradedAmbiguous

In [ ]:
discussionForumsPostsData['course_topic'] = discussionForumsPostsData['metadata_context_id'] + "-" + discussionForumsPostsData['body_discussion_topic_id']
discussionForumsPostsData

In [ ]:
discussionForumsPostsData = discussionForumsPostsData.loc[~discussionForumsPostsData['course_topic'].isin(courseTopicGradedUngradedAmbiguous['course_topic'])]
discussionForumsPostsData

In [ ]:
courseTopicGradedUngradedList = courseTopicGradedUngradedCheck.loc[~(courseTopicGradedUngradedCheck['course_topic'].isin(courseTopicGradedUngradedAmbiguous['course_topic']))]
courseTopicGradedUngradedList

In [ ]:
courseTopicGradedUngradedList.to_csv("CourseTopicGradedUngradedList_Intercession_2022.csv", header = True, index = False)

In [ ]:
discussionForumsPostsDataUngraded = discussionForumsPostsData.loc[discussionForumsPostsData['is_graded'] == False]
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded.drop(['is_graded'], axis = 1, inplace = True)
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded['metadata_event_time'] = discussionForumsPostsDataUngraded['metadata_event_time'].apply(str)
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded = discussionForumsPostsDataUngraded.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].apply(list).reset_index()
discussionForumsPostsDataUngraded

In [ ]:
discussionForumsPostsDataUngraded.to_csv("UngradedDiscussionForumsPosts_Intercession_2022.csv", header = True, index = False)

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsData.loc[discussionForumsPostsData['is_graded'] == True]
discussionForumsPostsDataGraded.drop(['is_graded'], axis = 1, inplace = True)
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded['metadata_event_time'] = discussionForumsPostsDataGraded['metadata_event_time'].apply(str)
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded = discussionForumsPostsDataGraded.groupby(['metadata_context_id', 'metadata_user_id'])['metadata_event_time'].apply(list).reset_index()
discussionForumsPostsDataGraded

In [ ]:
discussionForumsPostsDataGraded.to_csv("GradedDiscussionForumsPosts_Intercession_2022.csv", header = True, index = False)